In [1]:
#importar librerias

import numpy as np
import pandas as pd
from shutil import copyfile
from tensorflow.compat.v1 import ConfigProto, InteractiveSession
import split_folders
from PIL import ImageFile, Image
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.pyplot import figure
import matplotlib.image as mpimg
from matplotlib.patches import Circle, Rectangle
import os
from os.path import isfile, join
from os import listdir
from cv2 import imread, imwrite, imshow, waitKey, destroyAllWindows, CascadeClassifier, rectangle, cvtColor, COLOR_RGB2BGR
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from fuzzywuzzy import fuzz
from IPython.display import clear_output
import sys
import ipywidgets as widgets
from IPython.display import display
import random
from scipy import ndarray
# image processing library
import skimage as sk
from skimage import transform, util, io, img_as_uint

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/home/workstation/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/workstation/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/workstation/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/workstation/anaconda3/env

In [2]:
#setear carpetas
DATA1 = "/home/workstation/Desktop/fv/data/pics/pic1/"
DATA2 = "/home/workstation/Desktop/fv/data/pics/pic2/"
#TOP_DATA = "/home/workstation/Desktop/fv/topwanted/"
ALL_DATA = "/home/workstation/Desktop/fv/allwanted/"
LOWCOUNT = "/home/workstation/Desktop/fv/lowcount/"
FACES = '/home/workstation/Desktop/fv/faces/'
FIT_DATA ="/home/workstation/Desktop/fv/fit/"
val_dir = "/home/workstation/Desktop/fv/fit/val/"
train_dir = "/home/workstation/Desktop/fv/fit/train/"
test_dir = "/home/workstation/Desktop/fv/fit/test/"
TOP_DATA = "/home/workstation/Desktop/fv/spread/"
carpetas = [ DATA1, DATA2, TOP_DATA, ALL_DATA, LOWCOUNT, FACES, FIT_DATA,val_dir ,train_dir ,test_dir]

for carpeta in carpetas:
    if not os.path.exists(carpeta):
        os.mkdir(carpeta)
        
predict = '/home/workstation/Desktop/fv/perdictions/preds/'
crops = '/home/workstation/Desktop/fv/predictions/crops/'

In [3]:
#creamos una lista de nombres
df_faces = pd.read_csv("/home/workstation/Desktop/fv/data/DB_cleaned.csv",encoding="utf-8", index_col=0)
df_faces = pd.DataFrame(df_faces)
mask = (df_faces['fecha_hora_registro'] > '2015-01-01 01:39:44') & (df_faces['fecha_hora_registro'] <= '2020-03-30 10:39:44')
df_faces = df_faces.loc[mask]
df_faces_nombre = df_faces.groupby(["nombre"])
max_names2 = len(df_faces_nombre)
print(max_names2)

/home/workstation/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


204229


In [4]:
top_names = df_faces_nombre.count().sort_values(by=['foto1'], ascending=False).iloc[0:20000]["foto1"].index
all_names = df_faces_nombre.count().sort_values(by=['foto1'], ascending=False).iloc[0:max_names2]["foto1"].index
top_names

Index(['CESAR OMAR DELGADO SILVA ', 'JOSE GUADALUPE NEGRETE CAMACHO',
       'ROGELIO MORENO LOPEZ', 'JOSE GUADALUPE GONZALEZ MENDEZ',
       'FRANCISCO JAVIER MUÑOZ LOPEZ', 'ROBERTO CARLOS ROMERO NEGRETE',
       'JOSE RICARDO  ALVARADO FLORES ', 'ULISES MIGUEL MANRIQUEZ TORRES',
       'DANIEL OLVERA MUÑOZ', 'JUAN ANTONIO JUAREZ MADRIGAL',
       ...
       'LUIS ALEJANDRO REYES BARRIENTOS', 'JOSUE ALFREDO GONZALEZ RODRIGUEZ',
       'FRANCISCO JAVIER X ALVIZO', 'JARED SEBASTIAN LEDEZMA TORRES',
       'MIGUEL ALEJANDRO CABELLO SANCHEZ', 'ROQUE RUBIO HERNANDEZ',
       'JESUS MAURICIO VARGAS CRUZ', 'CHRISTIAN ISAAC RIVERA ESCAREÑO',
       'JOSE ESTEBAN CARMONA ALVAREZ', 'JESUS CONTRERAS PUGA'],
      dtype='object', name='nombre', length=20000)

In [5]:
df_faces = pd.read_csv("/home/workstation/Desktop/fv/data/DB_cleaned.csv",encoding="utf-8", index_col=0)
df_faces = pd.DataFrame(df_faces)
df_faces_nombre = df_faces.groupby(["nombre"])
max_names2 = len(df_faces_nombre)


In [6]:
#crear carpeta por persona y copiar fotos de cada persona

final_val = len(top_names)
progress = widgets.IntProgress(value=0, min=0, max=final_val)
display(progress)
for name in top_names:
    fotos1 = df_faces_nombre.get_group(name)["foto1"]
    fotos1 = pd.Series.to_frame(fotos1)
    fotos2 =  df_faces_nombre.get_group(name)["foto2"]
    fotos2 = pd.Series.to_frame(fotos2)
    progress.value +=1
    try:
        os.mkdir(TOP_DATA + name)   
        for foto in fotos1['foto1']:
            if type(foto) == str and os.path.exists(DATA1+foto):
            
                try:
                    copyfile(DATA1+foto, TOP_DATA + name + '/' + foto)
                except:
                    print(foto)
        for foto in fotos2['foto2']:
            if type(foto) == str and os.path.exists(DATA2+foto):
                try:
                    copyfile(DATA2+foto, TOP_DATA + name + '/' + foto)
                except:
                    print(foto)
    except:
        pass
#aqui ponemos otra barra de progreso

IntProgress(value=0, max=20000)

In [7]:
#crear carpeta por persona y copiar fotos de cada persona/// se nececita esto??
final_val = len(all_names)
progress = widgets.IntProgress(value=0, min=0, max=final_val)
display(progress, progress.value)
for name in all_names:
    progress.value +=1
    fotos1 = df_faces_nombre.get_group(name)["foto1"]
    fotos1 = pd.Series.to_frame(fotos1)
    fotos2 =  df_faces_nombre.get_group(name)["foto2"]
    fotos2 = pd.Series.to_frame(fotos2)
    try:
        os.mkdir(ALL_DATA + name)   
        for foto in fotos1['foto1']:
            if type(foto) == str and os.path.exists(DATA1+foto):
            
                try:
                    copyfile(DATA1+foto, ALL_DATA + name + '/' + foto)
                except:
                    print(foto)
        for foto in fotos2['foto2']:
            if type(foto) == str and os.path.exists(DATA2+foto):
                try:
                    copyfile(DATA2+foto, ALL_DATA + name + '/' + foto)
                except:
                    print(foto)
    except:
        pass
    #creo que se puede hacer mas rapido quitando los dos for y dejando solo 1

IntProgress(value=0, max=95159)

0